In [34]:
from sklearn.model_selection import train_test_split
from modules.classification_model_skeletons import *
from modules.data_preprocessing import *


In [35]:
df = load_data_into_df('data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'QPU'])

In [60]:
df_testing = filter_by_label(df, ['binary_number', 'QPU'])
df_testing = df_testing[:40000]
df_testing

,binary_number,QPU
0,0001011100100011110100100001101111000011010101...,ibm_brisbane
1,0110101010000001110100000100111101101111110111...,ibm_brisbane
2,1001001010000000001111010010111000100011110000...,ibm_brisbane
3,0101100101110010001001011000011010111011101101...,ibm_brisbane
4,1000101001000000111110011000010111010011111110...,ibm_brisbane
...,...,...
39995,0010010110101100011000101110101101010110000011...,ibm_sherbrooke
39996,1101001111000011101100001011011000010001100011...,ibm_sherbrooke
39997,0010000101001010011001111001100100011011100000...,ibm_sherbrooke
39998,0010110101100001001111101000111100010111111000...,ibm_sherbrooke


In [61]:
df_testing_concatenated = concatenate_data(df_testing, 10)
df_testing_concatenated

,Concatenated_Data,QPU
0,0001011100100011110100100001101111000011010101...,ibm_brisbane
1,1011101100100101010010110011110011110100100000...,ibm_brisbane
2,0001100011000111101001010110010011110111010100...,ibm_brisbane
3,0100011000001000101001100000011010010001101101...,ibm_brisbane
4,1011011110011000000100011001001101111111001010...,ibm_brisbane
...,...,...
3995,1110000110010111011111010000011011101111101011...,ibm_sherbrooke
3996,0111010110101000010000101100001000111100001001...,ibm_sherbrooke
3997,0000000110011000010011101010000100111110010000...,ibm_sherbrooke
3998,1110000100010100001110111110011110001100000011...,ibm_sherbrooke


In [62]:
tests = ['counts', 'unq_subsq', 'runs']
df_testing_features = apply_individual_qubit_functions(df_testing_concatenated, tests)
df_testing_features

/Users/shrey/Dev/qc-classifier-experiments/modules/data_preprocessing.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{function}_qb_{qb}'] = qubitRes
/Users/shrey/Dev/qc-classifier-experiments/modules/data_preprocessing.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{function}_qb_{qb}'] = qubitRes
/Users/shrey/Dev/qc-classifier-experiments/modules/data_preprocessing.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor p

,Concatenated_Data,QPU,counts_qb_0,counts_qb_1,counts_qb_2,counts_qb_3,counts_qb_4,counts_qb_5,counts_qb_6,counts_qb_7,...,runs_qb_90,runs_qb_91,runs_qb_92,runs_qb_93,runs_qb_94,runs_qb_95,runs_qb_96,runs_qb_97,runs_qb_98,runs_qb_99
0,0001011100100011110100100001101111000011010101...,ibm_brisbane,6,6,5,6,4,4,4,5,...,7,5,7,7,7,4,5,5,5,4
1,1011101100100101010010110011110011110100100000...,ibm_brisbane,5,6,5,1,5,6,2,3,...,7,7,7,7,4,5,6,3,4,6
2,0001100011000111101001010110010011110111010100...,ibm_brisbane,3,8,6,3,6,7,7,4,...,4,3,4,6,5,6,6,6,4,6
3,0100011000001000101001100000011010010001101101...,ibm_brisbane,4,5,6,6,5,6,7,6,...,6,5,5,7,5,8,7,4,8,3
4,1011011110011000000100011001001101111111001010...,ibm_brisbane,6,6,3,5,4,8,4,6,...,6,6,5,3,5,8,6,3,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1110000110010111011111010000011011101111101011...,ibm_sherbrooke,4,3,6,7,5,8,6,6,...,5,3,6,3,6,4,9,7,5,6
3996,0111010110101000010000101100001000111100001001...,ibm_sherbrooke,8,4,2,5,5,4,3,4,...,8,9,4,4,7,6,5,4,6,4
3997,0000000110011000010011101010000100111110010000...,ibm_sherbrooke,8,7,7,7,5,7,3,4,...,7,5,6,5,6,4,6,4,5,8
3998,1110000100010100001110111110011110001100000011...,ibm_sherbrooke,6,2,5,6,3,6,6,5,...,6,3,7,6,5,6,3,5,5,8


In [63]:
#df_testing_features = make_bit_features(df_testing_concatenated)
#df_testing_features

df_testing_features = df_testing_features.drop(columns='Concatenated_Data')
df_testing_features

,QPU,counts_qb_0,counts_qb_1,counts_qb_2,counts_qb_3,counts_qb_4,counts_qb_5,counts_qb_6,counts_qb_7,counts_qb_8,...,runs_qb_90,runs_qb_91,runs_qb_92,runs_qb_93,runs_qb_94,runs_qb_95,runs_qb_96,runs_qb_97,runs_qb_98,runs_qb_99
0,ibm_brisbane,6,6,5,6,4,4,4,5,7,...,7,5,7,7,7,4,5,5,5,4
1,ibm_brisbane,5,6,5,1,5,6,2,3,5,...,7,7,7,7,4,5,6,3,4,6
2,ibm_brisbane,3,8,6,3,6,7,7,4,4,...,4,3,4,6,5,6,6,6,4,6
3,ibm_brisbane,4,5,6,6,5,6,7,6,8,...,6,5,5,7,5,8,7,4,8,3
4,ibm_brisbane,6,6,3,5,4,8,4,6,2,...,6,6,5,3,5,8,6,3,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,ibm_sherbrooke,4,3,6,7,5,8,6,6,4,...,5,3,6,3,6,4,9,7,5,6
3996,ibm_sherbrooke,8,4,2,5,5,4,3,4,6,...,8,9,4,4,7,6,5,4,6,4
3997,ibm_sherbrooke,8,7,7,7,5,7,3,4,3,...,7,5,6,5,6,4,6,4,5,8
3998,ibm_sherbrooke,6,2,5,6,3,6,6,5,6,...,6,3,7,6,5,6,3,5,5,8


In [65]:
X = df_testing_features.drop(columns=df_testing_features.columns[0]).values
y = df_testing_features[df_testing_features.columns[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
sk_learn_gradient_boosting(X_train, y_train, X_test, y_test)

Gradient Boosting Accuracy: 0.9225


#### Basic MLP model

In [68]:
from sklearn.neural_network import MLPClassifier

mlpModel = MLPClassifier(hidden_layer_sizes=(250,), random_state=42, max_iter=800)

print('fitting model')
mlpModel.fit(X_train, y_train)

print('model fitted, now making predictions on test data')
model_y_predict = mlpModel.predict(X_test)

accuracy_mlp = accuracy_score(y_test, model_y_predict)

print('accuracy_score: ', accuracy_mlp)

fitting model
model fitted, now making predictions on test data
accuracy_score:  0.905
